<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Создание-класса" data-toc-modified-id="Создание-класса-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Создание класса</a></span></li><li><span><a href="#Моделирование" data-toc-modified-id="Моделирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Моделирование</a></span></li></ul></div>

# Описание

Данная программа моделирует гравитационную систему N материальных точек с произвольными массами, начальными координатами и начальными скоростями. Для исключения удаления материальных точек на бесконечность от центра экрана созданы барьеры за пределами видимости.

# Реализация

## Импорт библиотек

In [17]:
pip install screeninfo

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [18]:
from screeninfo import get_monitors
import graphics as gr
import numpy as np
import pandas as pd
from numpy.linalg import norm
import os

## Создание класса

In [19]:
class GrSystem:
    
    def __init__(self, count=15, G=5000, dt=0.01, size_x=None, size_y=None):
        self.count = count  # Number of bodies
        self.G = G          # Gravitational constant
        self.dt = dt        # Time step
        self.ctr = 0        # Controlled body number
        
        for m in get_monitors():
            if m.is_primary:
                if size_x == None:
                    self.size_x = m.width
                else:
                    self.size_x = size_x
                if size_y == None:
                    self.size_y = m.height
                else:
                    self.size_y = size_y

        self.txt = gr.Text(gr.Point(self.size_x / 6, self.size_y / 6), '')
        self.txt.setTextColor('white')
        self.txt.setSize(36)
        
        return
    
    
    def create_window(self, color='black'):

        window = gr.GraphWin('Гравитационная система', self.size_x, self.size_y)
        window.setBackground(color)
        self.window = window
        return
        
    
    def create_data(self, mass_list=None, coords_list=None, vel_list=None, col_list=None):

        if mass_list == None:
            mass_list = self.random_mass()
        if coords_list == None:
            coords_list = self.random_coords()
        if vel_list == None:
            vel_list = self.random_vels()
        if col_list == None:
            col_list = self.random_colors()
                
        self.a_mass = mass_list
        self.a_acc = np.zeros((self.count, 2))
        self.a_vel = vel_list
        self.a_coords = coords_list
        self.a_color = col_list
        self.a_radius = self.calculate_radiuses(mass_list)
        self.a_gr_obj = self.create_obj(coords_list, self.a_radius, col_list)

        return
    
    
    def create_obj(self, coords, radiuses, colors):
        
        result = pd.Series(np.empty(self.count))
        for i in range(len(radiuses)):
            result[i] = gr.Circle(gr.Point(*coords[i]), radiuses[i])
            result[i].setFill(colors[i])
            result[i].draw(self.window)
        self.txt.draw(self.window)
        return result
    
    
    def calculate_radiuses(self, series, min_rad=5):

        min_mass = series.min()

        return pd.Series(min_rad * (series / min_mass)**(1/3))

    def random_mass(self):

        array = np.random.normal(loc=4, scale=1, size=self.count)
        array[0] *= 20000; array[1] *= 500                                           # pass
        return np.abs(array / array.max())


    def random_coords(self):

        array = np.random.randn(self.count, 2)
        array /= (abs(array).max()+0.2)
        array[0] = np.array([0, 0])                                               # pass
        
        array[:,0] = self.size_x * (array[:,0] / 2 + 0.5)
        array[:,1] = self.size_y * (array[:,1] / 2 + 0.5)
        
        return array
    
    
    def random_vels(self, scale=10) -> np.ndarray:
        
        result = scale*np.random.randn(self.count, 2)
        result[0] *= 0                                                            # pass
        
        return result
    
    
    def random_colors(self) -> list:
    
        colors = ['yellow', 'red', 'lime', 'white', 'blue', 'orange', 'purple', 'pink', 'mediumspringgreen']
        
        return list(np.random.choice(colors, self.count))
    
    
    def acc_upd(self):
        
        for i in range(self.count):
            coords = self.a_coords[i]
            dist = np.append(self.a_coords[:i], self.a_coords[i+1:]).reshape(-1, 2) - coords
            
            self.a_acc[i] = (((self.G * np.append(self.a_mass[:i], self.a_mass[i+1:])).reshape(-1, 1) /
                              np.power(np.power(dist[:,0], 2) + np.power(dist[:,1], 2), 1.5).reshape(-1, 1)
                              * dist).sum(axis=0))
        
        return
    
    
    def vel_upd(self):
        
        self.a_vel += self.a_acc * self.dt
        
        return
    
    
    def coords_upd(self):
        
        self.coords_deltas = self.a_vel * self.dt
        self.a_coords += self.coords_deltas
        
        return
    
    
    def gr_obj_upd(self):
        
        for i in range(self.count):
            self.a_gr_obj[i].move(*self.coords_deltas[i])
            
        return
    
    
    def bounds(self, k=0.5, slowing=1):
        
        size = [self.size_x, self.size_y]
        
        for i in range(2):
            mask = self.a_coords[:, i] < -k * size[i]
            self.a_vel[mask, i] = abs(self.a_vel[mask, i]) * slowing
            mask = self.a_coords[:, i] > (1 + k) * size[i]
            self.a_vel[mask, i] = -abs(self.a_vel[mask, i]) * slowing
        
        return
    
    
    def restriction(self, bound_v=70, bound_a=0.2, slowing=0.99):
        
        mask = ((norm(self.a_vel, axis=1) > bound_v) * (norm(self.a_acc, axis=1) < bound_a)
                + norm(self.a_vel, axis=1) > 8*bound_v)
        self.a_vel[mask,:] *= slowing
        
        mask = norm(self.a_acc, axis=1) > bound_a*5000
        self.a_acc[mask,:] = 0
        
        return
    
    
    def control(self):
        
        k = self.window.checkKey()
        if k != '':
            print(f'{k = }', end=100*' '+'\r')
        if k == 'Control_R':
            if self.ctr == self.count - 1:
                self.ctr = 0
            else:
                self.ctr += 1
            self.txt.setText(f'--- ctr_r\tBody = {self.ctr}')
            print(f'--- ctr_r\tBody = {self.ctr}', end=100*' '+'\r')
        if k == 'Control_L':
            if self.ctr == 0:
                self.ctr = self.count - 1
            else:
                self.ctr -= 1
            self.txt.setText(f'--- ctr_l\tBody = {self.ctr}')
            print(f'--- ctr_l\tBody = {self.ctr}', end=100*' '+'\r')
        elif k == 'Up':
            self.dt *= 1.05
            self.txt.setText(f'--- Up\tdt = {self.dt:.4f}')
            print(f'--- Up\tdt = {self.dt:.4f}', end=100*' '+'\r')
        elif k == 'Down':
            self.dt *= 0.95
            self.txt.setText(f'--- Down\tdt = {self.dt:.4f}')
            print(f'--- Down\tdt = {self.dt:.4f}', end=100*' '+'\r')
        elif k == 'Escape': # Прекращение моделирования по нажатию клавиши 'Escape'
            self.window.close()
            self.txt.setText(f'--- Escape')
            print(f'--- Escape', end=100*' '+'\r')
            return True
        elif k == 'a':
            self.a_acc[self.ctr, 0] -= 100
            self.txt.setText(f'--- a')
            print(f'--- a', end=100*' '+'\r')
        elif k == 'w':
            self.a_acc[self.ctr, 1] -= 100
            self.txt.setText(f'--- w')
            print(f'--- w', end=100*' '+'\r')
        elif k == 'd':
            self.a_acc[self.ctr, 0] += 100
            self.txt.setText(f'--- d')
            print(f'--- d', end=100*' '+'\r')
        elif k == 's':
            self.a_acc[self.ctr, 1] += 100
            self.txt.setText(f'--- s')
            print(f'--- s', end=100*' '+'\r')
        elif k == 'space':
            self.a_vel[self.ctr,:] *= 0
            self.txt.setText(f'--- space')
            print(f'--- space', end=100*' '+'\r')
        
        return False


def prepare_data():
    with open('data_a.csv', 'w') as f:
        f.write('acceleration\n')
    with open('data_v.csv', 'w') as f:
        f.write('velocity\n')
    with open('data_c.csv', 'w') as f:
        f.write('X,Y\n')
    
    return


def write_data():
    pd.Series(norm(system.a_acc, axis=1), dtype='float64').to_csv('data_a.csv', mode='a', index=False, header=False)
    pd.Series(norm(system.a_vel, axis=1), dtype='float64').to_csv('data_v.csv', mode='a', index=False, header=False)
    pd.DataFrame(system.a_coords, dtype='float64').to_csv('data_c.csv', mode='a', index=False, header=False)
    
    return

## Моделирование

In [20]:
system = GrSystem(count=30, G=100000, dt=0.02)
system.create_window()
system.create_data()    # Задание параметров и их запись в df

# prepare_data() # <-- Раскомментировать для записи данных в файлы
i=0
while True:
    
                        # Вычисления:
    system.acc_upd()    # ускорений (и их запись в self.a_acc)
    if system.control():
        break
    system.vel_upd()    # скоростей (и их запись в self.a_vel)
    if system.control():
        break
    system.coords_upd() # координат (и их запись в self.a_coords)
    
    system.gr_obj_upd()    # Создание (отрисовка) на основе параметров графиеских объектов и их запись в df
    system.bounds() # Условие отражения объектов от границ 
    system.restriction(bound_v=10) # Ограничение скорости объектов
    
    # write_data() # <-- Раскомментировать для записи данных в файлы
    


C:\Users\Iren\AppData\Local\Temp\ipykernel_2964\367555454.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Circle(Point(683.0, 384.0), 150.85221882771495)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result[i] = gr.Circle(gr.Point(*coords[i]), radiuses[i])
